<a href="https://colab.research.google.com/github/jaunty0216/Final_Project_forNTNU/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E5%B0%88%E9%A1%8C%E3%80%80%E3%80%90%E4%BA%8C%E6%9C%88%E3%80%91%E6%83%85%E7%B7%92%E6%8A%92%E7%99%BC%E5%B0%8F%E5%A4%A9%E5%9C%B0%E3%80%80%E6%96%87%E5%AD%97%E9%9B%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

【二月】情緒抒發小天地文字探勘

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1pfMPJtcdJy0aNbBg4wATNVF7mlctQDgeaGk4HMLb1Ik/edit?resourcekey=&gid=2012234167#gid=2012234167').sheet1

In [3]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1pfMPJtcdJy0aNbBg4wATNVF7mlctQDgeaGk4HMLb1Ik/edit?resourcekey=&gid=2012234167#gid=2012234167').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [4]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.head()

,時間戳記,這個月有什麼煩惱呢？
0,,今天什麼都不想做，只想窩著。
1,,心裡像打結一樣，理不清楚。
2,,有點累，但還是撐過來了。
3,,好想逃到一個沒人認識的地方。
4,,天氣晴朗，心也輕了些。


In [6]:
df.describe()

,時間戳記,這個月有什麼煩惱呢？
count,30,30
unique,1,30
top,,今天什麼都不想做，只想窩著。
freq,30,1


In [7]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=65dc4637d8c8916245aa2f421916218299bd75b01e3370d0fd0deb50d9098ae2
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [8]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [9]:
for _, row in dicts.iterrows():
    word = row['這個月有什麼煩惱呢？']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.632 seconds.
DEBUG:jieba:Loading model cost 1.632 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [10]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [11]:
df['Chinese_Content'] = df['這個月有什麼煩惱呢？'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [12]:
df['Chinese_Content']

,Chinese_Content
0,今天什麼都不想做只想窩著
1,心裡像打結一樣理不清楚
2,有點累但還是撐過來了
3,好想逃到一個沒人認識的地方
4,天氣晴朗心也輕了些
5,突然好想念以前無憂無慮的日子
6,混亂不安沒方向
7,好像慢慢開始懂得放下了
8,今天有被一句話療癒到
9,我值得被善待也正在學著善待自己


In [13]:
df['Tokenized']

,Tokenized
0,"[今天, 什麼, 都, 不想, 做, 只, 想, 窩, 著]"
1,"[心裡, 像, 打結, 一樣理, 不, 清楚]"
2,"[有點, 累, 但, 還是, 撐, 過來, 了]"
3,"[好想, 逃到, 一個, 沒人, 認識, 的, 地方]"
4,"[天氣, 晴朗, 心, 也, 輕, 了, 些]"
5,"[突然, 好, 想念, 以前, 無憂, 無慮, 的, 日子]"
6,"[混亂, 不安, 沒, 方向]"
7,"[好像, 慢慢, 開始, 懂得, 放下, 了]"
8,"[今天, 有, 被, 一句, 話療, 癒, 到]"
9,"[我, 值得, 被, 善待, 也, 正在, 學著, 善待, 自己]"


In [14]:
word_counts = Counter(all_words)
word_counts

Counter({'今天': 5,
         '什麼': 2,
         '不想': 2,
         '心裡': 3,
         '打結': 1,
         '一樣理': 1,
         '清楚': 1,
         '有點': 2,
         '還是': 1,
         '過來': 1,
         '好想': 1,
         '逃到': 1,
         '一個': 1,
         '沒人': 1,
         '認識': 1,
         '地方': 1,
         '天氣': 1,
         '晴朗': 1,
         '突然': 2,
         '想念': 1,
         '以前': 1,
         '無憂': 1,
         '無慮': 1,
         '日子': 1,
         '混亂': 1,
         '不安': 1,
         '方向': 1,
         '好像': 3,
         '慢慢': 2,
         '開始': 2,
         '懂得': 1,
         '放下': 1,
         '一句': 1,
         '話療': 1,
         '值得': 1,
         '善待': 2,
         '正在': 1,
         '學著': 1,
         '自己': 1,
         '太陽': 1,
         '溫暖': 1,
         '一切': 1,
         '還會': 1,
         '空氣': 1,
         '濕濕': 1,
         '心情': 1,
         '點潮': 1,
         '出來': 1,
         '焦慮': 1,
         '別人': 1,
         '看見': 1,
         '平靜': 1,
         '下午': 1,
         '心無波': 1,
         '一切都在': 1,
      

In [23]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個', '得','今天','開始','什麼','好像'
])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [24]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [27]:
word_freq_df

,Word,Frequency
0,心裡,3
1,不想,2
2,有點,2
3,善待,2
4,慢慢,2
...,...,...
101,只想,1
102,安靜過,1
103,疲憊,1
104,還在,1


In [28]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-wqy-zenhei is already the newest version (0.9.45-8).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [29]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [20]:
from google.colab import userdata
apikey = userdata.get('key')

In [21]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言主要呈現出負面情緒與積極情緒的交雜，但**負面情緒佔大多數**，風向偏向**低落、壓力大，但同時也伴隨著自我療癒和對未來的期許**。

**最多人呈現的情緒：低落、壓力、焦慮**

相關內容：

* **壓抑與無力感:**  許多留言表達了內心的壓抑和無力感，例如「心裡像打結一樣，理不清楚」、「好想逃到一個沒人認識的地方」、「混亂、不安、沒方向」、「想哭，卻哭不出來」、「一切都在掌控之外，卻還要假裝沒事」、「心裡空空的，像被掏空了」、「害怕再受傷，所以什麼都不說」、「想被理解，卻又怕太赤裸」。這些表達都透露出負面情緒，以及無法宣洩的困境。
* **疲憊與倦怠:**  「有點累，但還是撐過來了」、「有些疲憊，但我還在，那就夠了」、「今天什麼都不想做，只想窩著」等留言展現了身心俱疲的狀態。
* **孤獨與失落:** 「突然好想念以前無憂無慮的日子」、「慢慢地，我開始習慣孤單了」、「心裡空空的，像被掏空了」顯示出對過去的懷念和對現在孤獨狀態的接受，也暗示著一種失落感。
* **焦慮與不安:** 「很焦慮，但不想讓別人看見」直接表達了焦慮的情緒，並帶有隱藏情緒的意圖。


**積極情緒及自我療癒：**

雖然負面情緒佔據主導，但仍有一些留言展現了積極的一面和自我療癒的過程：

* **尋求慰藉與自我調節：**  「今天有被一句話療癒到」、「一杯熱茶，一本書，今天只想安靜過」、「太陽好溫暖，好像一切都還會好」、「一點點希望，就能照亮一整天」、「心裡的雲終於散了，呼吸變得輕盈」、「我值得被善待，也正在學著善待自己」等留言顯示了他們正在嘗試尋求慰藉，並透過自我調節來面對困境。
* **對未來的期許：** 「雖然辛苦，但有點期待未來」展現了即使處於困境，仍然抱持著對未來的希望。


總體而言，這些留言反映了現代人普遍存在的壓力與焦慮，但同時也展現了他們在困境中積極尋求解脫和自我療癒的韌性。  他們渴望被理解，卻也害怕暴露脆弱的一面。  整體風向是低落但並非絕望，帶有自我療癒和對未來的期許。

